In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
from mlflow.tracking import MlflowClient
from azure.ai.ml import MLClient#, Input, command
from azure.identity import DefaultAzureCredential
import sys
sys.path.append("../..")
from utils import azure_ml_configs

workspace_id = azure_ml_configs.workspace_id
subscription_id = azure_ml_configs.subscription_id
resource_group = azure_ml_configs.resource_group
workspace_name = azure_ml_configs.workspace_name

# Get a handle to the workspace
ml_client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

mlflow_tracking_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri


# Initialize MLflow client
client = MlflowClient(mlflow_tracking_uri)

# Get experiment by name
experiment_name = "mlm-jobs"
experiment = client.get_experiment_by_name(experiment_name)
#client.get_run(run_id)

experiment_id = experiment.experiment_id

runs = client.search_runs(experiment_id)[::-1]

In [ ]:
newest_run_id = runs[0].info.run_id
newest_run = client.get_run(newest_run_id)

# Get metrics for all steps
for metric_key in newest_run.data.metrics:
    metric_history = client.get_metric_history(newest_run_id, metric_key)
    print(f"\nMetric: {metric_key}")
    for metric in metric_history:
        print(f"Step: {metric.step}, Value: {metric.value}")

In [ ]:
# combine and make plot

ppl = []
train_loss = []
eval_loss = []

results = {"perplexity": [],
           "train_loss": [],
           "eval_loss": []
          }

runs_index = [3,2,1,0]

for i in runs_index:
    newest_run_id = runs[i].info.run_id
    newest_run = client.get_run(newest_run_id)

    # Get metrics for all steps
    for metric_key in results.keys():
        metric_history = client.get_metric_history(newest_run_id, metric_key)
        #print(f"\nMetric: {metric_key}")
        for metric in metric_history:
            results[metric_key].append(metric.value)

In [ ]:
sns.set_palette("mako_r",3)
sns.set_style("white", {"axes.edgecolor": ".8"})
palette = sns.color_palette("mako_r")

fig, ax = plt.subplots(1,1, figsize=(5,4))

line1 = ax.plot(np.arange(0,len(results["eval_loss"])),results["eval_loss"], "-o", label="Validation loss", color=palette[0]);
line2 = ax.plot(np.arange(0,len(results["train_loss"])),results["train_loss"], "-o", label="Training loss", color=palette[1]);
ax.axvline(x=2, ls="--", c="black", lw=0.7)
ax.axvline(x=5, ls="--", c="black", lw=0.7)
ax.axvline(x=8, ls="--", c="black", lw=0.7)
ax.set_ylabel('Cross-Entropy Loss')
ax.set_xlabel('Epoch')
plt.xticks(labels=[i for i in np.arange(1,13)],ticks=np.arange(0,12))

ax2 = ax.twinx()
line3 = ax2.plot(np.arange(0,len(results["perplexity"])),results["perplexity"], "-o", label="Perplexity", color=palette[3]);
ax2.set_ylabel('Perplexity') 

lns = line1+line2+line3
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc=0);

ax.tick_params(left = False)
ax2.tick_params(right=False)

ax.set_title("MLM performance");
plt.savefig("../../output/MLM_style2.pdf", bbox_inches="tight")
plt.savefig("../../output/MLM_style2.png", bbox_inches="tight")